In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Prompt Used to generate SaaS Training Dataset:

Can you help me generate a long list customer requests that come in for some Saas company that has to be triaged to different departments like product teams, support teams, finance teams etc. Could you make the requests or questions from customers as realistic as possible. Provide the answer in terms of the question asked and the team the request needs to be triaged to.

In [3]:
saas_data = "/Users/abhishekkarmakar/Desktop/TAGGING/GPT based triaging.txt"

In [4]:
requests, teams = [], []

with open(saas_data, "r") as file:
    lines = file.readlines()
    for line in lines:
        if line.startswith("Request:"):
            request = line.split("Request:", 1)[1].strip()
            request = request.strip('"') # This line removes the quotation marks
            requests.append(request)
        elif line.startswith("Team:"):
            teams.append(line.split("Team:", 1)[1].strip())            

In [5]:
df = pd.DataFrame({
    'Request': requests,
    'Team': teams
})

df['Team'].value_counts()

Support Team                460
Finance Team                459
Product Development Team    458
Sales Team                   17
Training Team                 6
Marketing Team                5
Legal Team                    5
Security Team                 4
IT Operations Team            4
Implementation Team           2
Finance                       1
Name: Team, dtype: int64

In [6]:
df = df[(df['Team'] == 'Support Team') | (df['Team'] == 'Finance Team') | (df['Team'] == 'Product Development Team')]

In [7]:
df['Team'].value_counts()

Support Team                460
Finance Team                459
Product Development Team    458
Name: Team, dtype: int64

In [8]:
df

,Request,Team
0,I'm having trouble logging into my account.,Support Team
1,How do I upgrade my subscription plan?,Finance Team
2,I noticed a bug in the latest software update.,Product Development Team
3,I can't find the feature to export my data. Can you help?,Product Development Team
4,I need assistance integrating the API into my website.,Support Team
5,My payment was declined. What could be the issue?,Finance Team
6,Is there a way to track user engagement on the dashboard?,Product Development Team
7,The application is running slow. Can you optimise it?,Product Development Team
8,I accidentally deleted some important data. Can you recover it?,Support Team
9,What payment methods do you accept for the subscription?,Finance Team


In [9]:
# !pip install tensorflow
# !pip install tensorflow_hub

In [10]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import tensorflow_hub as hub

from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold

In [11]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
use_model = hub.load(module_url)

In [12]:
X = df['Request']
y = df['Team']

In [13]:
X_encoded = list(use_model(X))
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.1, random_state=42)

### Support Vector Machine

In [180]:
param_grid = {
    'C': [5,6,7,8,9],
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

classifier = SVC()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(classifier, param_grid, cv=cv, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [5, 6, 7, 8, 9], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'sigmoid']},
             scoring='accuracy')

In [181]:
best_params = grid_search.best_params_ # Get the best parameters and estimator from the grid search
print(best_params)
best_classifier = grid_search.best_estimator_

{'C': 5, 'gamma': 'scale', 'kernel': 'rbf'}


In [182]:
best_classifier

SVC(C=5)

In [183]:
y_pred = best_classifier.predict(X_test) # Predict using the best classifier

In [184]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Best Parameters: {best_params}")
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

# classifier = SVC(C= 7, gamma= 'scale', kernel='rbf')
# classifier.fit(X_train, y_train)

# y_pred = classifier.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# report = classification_report(y_test, y_pred)

Best Parameters: {'C': 5, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 0.9130434782608695
Classification Report:
                          precision    recall  f1-score   support

            Finance Team       0.97      0.93      0.95        42
Product Development Team       0.89      0.96      0.92        49
            Support Team       0.89      0.85      0.87        47

                accuracy                           0.91       138
               macro avg       0.92      0.91      0.91       138
            weighted avg       0.91      0.91      0.91       138



In [195]:
# single_sentence = "Where is the option of new user in Nurturev?" # Support Team
single_sentence = "The main page of Home app is not working. Make the corrections to rectify it."
single_sentence_encoded = use_model([single_sentence])
predicted_label = best_classifier.predict(single_sentence_encoded)
print("Predicted Label:", predicted_label[0])

Predicted Label: Product Development Team


In [197]:
import joblib  # Use joblib to save and load models
joblib.dump(best_classifier, 'best_svc_model.pkl') # Save the best_classifier to a file

['best_svc_model.pkl']

In [ ]:
# Load the saved model
import joblib  # Use joblib to save and load models
loaded_model = joblib.load('best_svc_model.pkl')

# Single Sentence Prediction for production
# single_sentence = "There is some issue in the Home app. It stops as soon as it opens." # Product Development Team
single_sentence = "What are the different notification functionalities supported in the webapp?" # Product Development Team
# My workflow is not reflecting the dashboard despite saving it. Can you please help me out? # Support
# Page taking too time to load # Support

single_sentence_encoded = list(use_model([single_sentence]))
predicted_label = loaded_model.predict(single_sentence_encoded)
print("Predicted Label:", predicted_label[0])

### FEEDBACK based model correction.

In [151]:
import joblib
import numpy as np

def predict_and_retrain():
    global df
    print(len(df))
    
    model = joblib.load('best_svc_model2.pkl')
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
    use_model = hub.load(module_url)
    
    X = df['Request']
    y = df['Team']
    X_encoded = list(use_model(X))
    
    single_sentence = input("Please provide the sentence for prediction: ")
    single_sentence_encoded = list(use_model([single_sentence]))
    predicted_label = model.predict(single_sentence_encoded)[0]
    print("Predicted Label:", predicted_label)
    
    feedback = input(f"Was the predicted label '{predicted_label}' correct? (yes/no): ").strip().lower()
    
    if feedback == 'no':
        correct_label = input("Please provide the correct label: ").strip()
        
        # Create a new DataFrame with the new data and concatenate it with the original df
        new_data = {'Request': [single_sentence], 'Team': [correct_label]}
        new_df = pd.DataFrame(new_data)
        df = pd.concat([df, new_df], ignore_index=True)
        
        # Combine the single feedback instance with the full original training data
        X_combined = np.vstack([single_sentence_encoded, np.array(X_encoded)])
        y_combined = [correct_label] + list(y)
        
        model.fit(X_combined, y_combined)  # Retrain the model with the combined data
        
        joblib.dump(model, 'best_svc_model2.pkl')
        print("Model has been updated and saved!")
    else:
        print("Thank you for the feedback!")

predict_and_retrain()

1381
Please provide the sentence for prediction: How should I structure the data model in my Netsuite application such that the payload that I get from the integration suites the use case of finance teams to conduct audit?
Predicted Label: Support Team
Was the predicted label 'Support Team' correct? (yes/no): no
Please provide the correct label: Product Development Team
Model has been updated and saved!


In [84]:
original_model = joblib.load('best_svc_model.pkl')
print("Original Model Support Vectors:", len(original_model.support_vectors_))

retrained_model = joblib.load('best_svc_model2.pkl')
print("Retrained Model Support Vectors:", len(retrained_model.support_vectors_))

# retrained_model2 = joblib.load('best_svc_model3.pkl')
# print("Retrained Model Support Vectors:", len(retrained_model2.support_vectors_))

Original Model Support Vectors: 706
Retrained Model Support Vectors: 760


In [ ]:
## Assuming that I have a feedback_dataframe which is saved on getting a "no" as feedback.
feedback_df = pd.DataFrame(columns=['Request','Team']) # This will be saved based on negative feedback.

import joblib
import tensorflow_hub as hub

def retrain_on_feedback(feedback_df):
    model = joblib.load('best_svc_model.pkl')
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
    use_model = hub.load(module_url)

    # Encode the sentences from feedback_df
    X_feedback_encoded = list(use_model(feedback_df['Request']))
    y_feedback = feedback_df['Team'].values

    # Retrain the model on the feedback data
    model.fit(X_feedback_encoded, y_feedback)

    # Save the retrained model back to 'best_svc_model.pkl'
    joblib.dump(model, 'best_svc_model.pkl')
    print("Model has been retrained and saved based on user feedback!")

# Assuming feedback_df is already loaded
retrain_on_feedback(feedback_df)

# Re-training in this procedure would reduce the cases of overfitting on the batch.

# DQN_classifier for Reinforcement Learning

In [17]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
use_model = hub.load(module_url)

In [63]:
X = df['Request']
y = df['Team']

X_encoded = list(use_model(X))
X_encoded = np.array(X_encoded)

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.1, random_state=42)

In [64]:
class DQNClassifier:
    def __init__(self, input_dim, num_classes, gamma=0.95, alpha=0.01):
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.model = self._build_model()
        self.gamma = gamma
        self.alpha = alpha

    def _build_model(self):
        model = Sequential()
        model.add(Dense(128, input_dim=self.input_dim, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.num_classes, activation='linear'))
        model.compile(optimizer=Adam(lr=0.001), loss='mse')
        return model

    def _compute_val_loss(self, X_val, y_val):
        val_loss = 0
        for i, state in enumerate(X_val):
            target = self.model.predict(np.array([state]))
            action = y_val[i]
            val_loss += target[0][action]
        return val_loss / len(X_val)

    def train(self, X_train, y_train, X_val, y_val, epochs=100, batch_size=16, patience=5):
        best_val_loss = float('inf')
        no_improvement_epochs = 0

        for epoch in range(epochs):
            for i, state in enumerate(X_train):
                target = self.model.predict(np.array([state]))

                # Q-learning component
                next_state_q_values = self.model.predict(np.array([state]))
                max_next_q_value = np.max(next_state_q_values)
                action = y_train[i]
                reward = 1
                updated_q_value = (1 - self.alpha) * target[0][action] + self.alpha * (reward + self.gamma * max_next_q_value)
                target[0][action] = updated_q_value
                self.model.fit(np.array([state]), target, epochs=1, verbose=0, batch_size=batch_size)

            # Early stopping based on validation loss
            val_loss = self._compute_val_loss(X_val, y_val)
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                no_improvement_epochs = 0
            else:
                no_improvement_epochs += 1

            if no_improvement_epochs >= patience:
                print(f"Early stopping at epoch {epoch}")
                break
                
    def predict(self, X):
        # Using the trained model to make predictions
        predicted_q_values = self.model.predict(X)
        # Return the action with the highest Q-value for each sample
        return np.argmax(predicted_q_values, axis=1)                

In [65]:
# 1. Instantiate the DQNClassifier
dqn_classifier = DQNClassifier(input_dim=X_train.shape[1], num_classes=len(np.unique(y_encoded)))

# 2. Train the model
dqn_classifier.train(X_train, y_train, X_test, y_test, epochs=100, batch_size=16, patience=5)

1/1 [==============================] - 0s 6ms/step


1/1 [==============================] - 0s 5ms/step


1/1 [==============================] - 0s 5ms/step


1/1 [==============================] - 0s 6ms/step


1/1 [==============================] - 0s 5ms/step


...

1/1 [==============================] - 0s 5ms/step


1/1 [==============================] - 0s 5ms/step


1/1 [==============================] - 0s 6ms/step


1/1 [==============================] - 0s 6ms/step


1/1 [==============================] - 0s 5ms/step
Early stopping at epoch 14


In [66]:
# (Optionally) 3. Make predictions on new data
y_pred = dqn_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

5/5 [==============================] - 0s 633us/step
Accuracy: 0.5434782608695652
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.67      0.80        42
           1       0.00      0.00      0.00        49
           2       0.43      1.00      0.60        47

    accuracy                           0.54       138
   macro avg       0.48      0.56      0.47       138
weighted avg       0.45      0.54      0.45       138



/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Early Stopping for DQN-Classifier

In [54]:
# DQN Classifier with Early Stopping
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, classification_report

class DQNClassifier:
    def __init__(self, input_dim, num_classes):
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(128, input_dim=self.input_dim, activation='relu'))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(self.num_classes, activation='linear'))
        model.compile(optimizer=Adam(lr=0.0001), loss='mse')
        return model

    def train(self, X_train, y_train, epochs=100, batch_size=16):
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[early_stopping])

    def predict(self, X_test):
        return np.argmax(self.model.predict(X_test), axis=1)

In [55]:
dqn_classifier = DQNClassifier(input_dim=X_train.shape[1], num_classes=len(np.unique(y_encoded)))
dqn_classifier.train(X_train, y_train)

y_pred = dqn_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Epoch 1/100
78/78 [==============================] - 0s 950us/step - loss: 0.6998 - val_loss: 0.2405
Epoch 2/100
78/78 [==============================] - 0s 560us/step - loss: 0.1859 - val_loss: 0.1974
Epoch 3/100
78/78 [==============================] - 0s 534us/step - loss: 0.1551 - val_loss: 0.1981
Epoch 4/100
78/78 [==============================] - 0s 541us/step - loss: 0.1387 - val_loss: 0.1923
Epoch 5/100
78/78 [==============================] - 0s 562us/step - loss: 0.1291 - val_loss: 0.1791
Epoch 6/100
78/78 [==============================] - 0s 538us/step - loss: 0.1075 - val_loss: 0.1857
Epoch 7/100
78/78 [==============================] - 0s 536us/step - loss: 0.0920 - val_loss: 0.1766
Epoch 8/100
78/78 [==============================] - 0s 525us/step - loss: 0.0783 - val_loss: 0.1710
Epoch 9/100
78/78 [==============================] - 0s 528us/step - loss: 0.0629 - val_loss: 0.1720
Epoch 10/100
78/78 [==============================] - 0s 587us/step - loss: 0.0497 - val_lo

# Q-Learning in Reinforcement Learning

In [15]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
use_model = hub.load(module_url)

X = df['Request']
y = df['Team']

In [16]:
X_encoded = list(use_model(X))
X_encoded = np.array(X_encoded)

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y)

# 2. Split the data
X_train, X_test, y_train, y_test = train_test_split(X_encoded, encoded_labels, test_size=0.1, random_state=42)

In [18]:
# 3. Q-learning
n_states = len(X_train)
n_actions = 3  # 3 classes: Support Team, Finance Team, Product Development Team
alpha = 0.01  # learning rate
gamma = 0.5  # discount factor
epsilon = 0.2  # exploration rate

Q = np.zeros((n_states, n_actions))

def get_reward(true_label, predicted_label):
    return 1 if true_label == predicted_label else -1

In [49]:
for epoch in range(100):  # Number of training iterations
    for i, state in enumerate(X_train):
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.choice(n_actions)  # Explore
        else:
            action = np.argmax(Q[i, :])  # Exploit

        reward = get_reward(y_train[i], action)
        Q[i, action] = Q[i, action] + alpha * (reward + gamma * np.max(Q[i, :]) - Q[i, action])

# 4. Evaluation
y_pred = []
for i, state in enumerate(X_test):
    action = np.argmax(Q[i, :])  # Exploit only during evaluation
    y_pred.append(action)

    if action == y_test[i]:
        correct_predictions += 1

# Classification report
labels = list(label_encoder.classes_)
report = classification_report(y_test, y_pred, target_names=labels)
print(f"Report : {report}")

Report :                           precision    recall  f1-score   support

            Finance Team       0.33      0.38      0.36        42
Product Development Team       0.35      0.31      0.33        49
            Support Team       0.32      0.32      0.32        47

                accuracy                           0.33       138
               macro avg       0.33      0.34      0.33       138
            weighted avg       0.33      0.33      0.33       138



## Early Stopping Criteria using Q-Learning

In [28]:
# 1. Split the data into train, validation, and test sets
X_trainval, X_test, y_trainval, y_test = train_test_split(X_encoded, encoded_labels, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.1, random_state=42)

n_states = len(X_train)
Q = np.zeros((n_states, n_actions))

patience = 10  # Number of epochs with no improvement after which training will be stopped.
best_val_loss = float('inf')
no_improvement_epochs = 0

for epoch in range(100):  # Maximum number of training iterations
    # Training
    for i, state in enumerate(X_train):
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.choice(n_actions)
        else:
            action = np.argmax(Q[i, :])

        reward = get_reward(y_train[i], action)
        Q[i, action] = Q[i, action] + alpha * (reward + gamma * np.max(Q[i, :]) - Q[i, action])

    # Validation
    val_loss = 0
    for i, state in enumerate(X_val):
        action = np.argmax(Q[i, :])
        reward = get_reward(y_val[i], action)
        val_loss -= reward  # Negative reward is our loss

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1

    if no_improvement_epochs >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

Early stopping at epoch 10


In [34]:
# ... Continue with the evaluation on X_test as before.

# 4. Evaluation
y_pred = []
for i, state in enumerate(X_test):
    action = np.argmax(Q[i, :])  # Exploit only during evaluation
    y_pred.append(action)

    if action == y_test[i]:
        correct_predictions += 1

# Classification report
labels = list(label_encoder.classes_)
report = classification_report(y_test, y_pred, target_names=labels)
print(f"Report : {report}")

Report :                           precision    recall  f1-score   support

            Finance Team       0.24      0.26      0.25        42
Product Development Team       0.20      0.20      0.20        49
            Support Team       0.23      0.21      0.22        47

                accuracy                           0.22       138
               macro avg       0.23      0.23      0.23       138
            weighted avg       0.22      0.22      0.22       138



# RLHF - Reinforcement Learning Human Feedback

In [39]:
# saas_data = "/Users/abhishekkarmakar/Desktop/TAGGING/GPT based triaging.txt"

# requests, teams = [], []
# with open(saas_data, "r") as file:
#     lines = file.readlines()
#     for line in lines:
#         if line.startswith("Request:"):
#             request = line.split("Request:", 1)[1].strip()
#             request = request.strip('"') # This line removes the quotation marks
#             requests.append(request)
#         elif line.startswith("Team:"):
#             teams.append(line.split("Team:", 1)[1].strip())

# df = pd.DataFrame({
#     'Request': requests,
#     'Team': teams
# })


# tasks = [
#     {
#         "prompt":df['Request'][i],
#         "response":df['Team'][i]
#     }
#     for i in df.index
# ]

# # Saving the GPT-Traiging Data into json format
# with open('GPT based triaging.json', 'w') as f:
#     json.dump(tasks, f, indent=4)

[{'prompt': "I'm having trouble logging into my account.",
  'response': 'Support Team'},
 {'prompt': 'How do I upgrade my subscription plan?',
  'response': 'Finance Team'}]

In [4]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import pandas as pd
import json
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config        

/opt/homebrew/anaconda3/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [159]:
class SFTModel(GPT2LMHeadModel):
    def __init__(self):
        configuration = GPT2Config.from_pretrained('gpt2')
        super().__init__(config=configuration)
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.to(torch.device("cpu"))

    def compute_loss(self, prompt, response):
        entire_text = prompt + response
        context_dict = self.tokenizer(entire_text, return_tensors="pt", padding=True, truncation=True, max_length=1024)
        
        input_ids = context_dict["input_ids"].to(self.device)
        attention_mask = context_dict["attention_mask"].to(self.device)
        labels = input_ids.clone()
        
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs.loss
    
    def predict(self, sentence):
        with torch.no_grad():
            inputs = self.tokenizer(sentence, return_tensors="pt", truncation=True, max_length=1024, padding="max_length")
            input_ids = inputs["input_ids"].to(self.device)
            attention_mask = inputs["attention_mask"].to(self.device)

            # Generate outputs
            outputs = self(input_ids=input_ids, attention_mask=attention_mask)

            # Extract the logits for the last token in the sequence
            last_token_logits = outputs.logits[0, -1, :]

            # Softmax to get "pseudo-probabilities"
            pseudo_probs = torch.nn.functional.softmax(last_token_logits, dim=-1)
            print(pseudo_probs)
            print(len(pseudo_probs))

            # We will only consider the first three as valid class logits
            class_logits = pseudo_probs[:3] 

            # Find the argmax
            predicted_class_idx = torch.argmax(class_logits).item()

            # Decode or interpret the prediction based on your task
            predicted_response = self.decode_prediction(predicted_class_idx)

            return predicted_response

        
    def decode_prediction(self, idx):
        class_names = ["Support Team", "Finance Team", "Product Development Team"]

        # Check that the index is valid
        if 0 <= idx < len(class_names):
            return class_names[idx]
        else:
            raise ValueError(f"Unexpected class index: {idx}")
                    

In [160]:
class SFTDataset(Dataset):
        """Supervised Fine-Tuning        ------->>>>            Returns: prompt: str
                                                                         response: str """        
        def __init__(self, file_path):
            with open(file_path) as f:
                self.data = json.load(f)
        def __len__(self):
            return len(self.data)
        def __getitem__(self, idx):
            return self.data[idx]["prompt"], self.data[idx]['response']

In [161]:
def train_and_save_SFT_model(epochs=10, data_file="GPT based triaging.json"):
    model = SFTModel()
    dataset = SFTDataset(file_path="GPT based triaging.json")
#     dataset = SFTDataset(data_file)
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, betas=(0.9, 0.95))
    writer = SummaryWriter()
    batch_idx = 0
    
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}")
        for batch in tqdm(dataloader):
            prompt, response = batch
            loss = model.compute_loss(prompt, response)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            writer.add_scalar("SFT Model Loss/train", loss.item(), batch_idx)
            batch_idx += 1

    torch.save(model.state_dict(), "sft_model_params.pt")
    

# train_and_save_SFT_model()

In [14]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import pandas as pd
import json
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config        

class GPT2ForClassification(GPT2LMHeadModel):
    def __init__(self, num_labels, tokenizer):
        config = GPT2Config.from_pretrained('gpt2')
        config.num_labels = num_labels
        super(GPT2ForClassification, self).__init__(config=config)

        self.num_labels = num_labels
        self.classifier = nn.Linear(self.config.n_embd, self.num_labels)

        # Resize token embeddings in case of added tokens
        self.resize_token_embeddings(len(tokenizer))

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs[0]

        cls_output = hidden_states[:, 0, :]  
        logits = self.classifier(cls_output)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return (loss, logits) if loss is not None else logits
    

class ClassificationDataset(Dataset):
    def __init__(self, data_file):
        with open(data_file, "r") as f:
            self.data = json.load(f)

        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

        # Add [CLS] token as a special token and resize model embeddings
        special_tokens_dict = {'cls_token': '[CLS]', 'pad_token': '[PAD]'}
        self.tokenizer.add_special_tokens(special_tokens_dict)
        
        self.label_map = {
            "Support Team": 0,
            "Finance Team": 1,
            "Product Development Team": 2
        }        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = "[CLS] " + item["prompt"]
        inputs = self.tokenizer.encode_plus(text, truncation=True, padding="max_length", max_length=1024, return_tensors="pt")
        
        label_id = self.label_map.get(item['response'], -1)
        if label_id == -1:
            raise ValueError(f"Invalid response: {item['response']}")
        
        return inputs["input_ids"].view(-1, 1024), inputs["attention_mask"].view(-1, 1024), torch.tensor(label_id)  
    
def train_classification_model(epochs=10):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    special_tokens_dict = {'cls_token': '[CLS]'}
    tokenizer.add_special_tokens(special_tokens_dict)

    model = GPT2ForClassification(num_labels=3, tokenizer=tokenizer).to(torch.device("cpu"))
    dataset = ClassificationDataset(data_file="GPT based triaging.json")
    tokenizer = dataset.tokenizer
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, betas=(0.9, 0.95))

    for epoch in range(epochs):
        for batch in dataloader:
            input_ids, attention_mask, labels = batch
            loss, _ = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    torch.save(model.state_dict(), "classification_model_params.pt")

train_classification_model()

IndexError: index out of range in self

In [140]:
model_path = "sft_model_params.pt"  # adjust path if needed
model = SFTModel()
model.load_state_dict(torch.load(model_path))
model.to(torch.device("cpu"))

SFTModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=

In [141]:
# sentence = "Where is the option of new user in Nurturev?"  # Support team
# sentence = "I can't access my account." # Support team
# sentence = "The main page of Home app is not working. Make the corrections to rectify it." # Product Development Team
sentence = "There is some issue in the Home app. It stops as soon as it opens." # Product Development Team
prediction = model.predict(sentence)
print("Predicted Response:", prediction)

tensor([2.8812e-07, 4.3842e-09, 4.3782e-09,  ..., 5.3221e-09, 2.8879e-09,
        9.9974e-01])
50257
Predicted Response: Support Team


In [142]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)  # you might want to adjust learning rate

def fine_tune_on_feedback(prompt, correct_response, epochs=1):
    # Convert your single example into a format suitable for DataLoader
    feedback_dataset = [{"prompt": prompt, "response": correct_response}]
    feedback_loader = DataLoader(feedback_dataset, batch_size=1, shuffle=True)

    model.train()  # Set the model to training mode
    for epoch in range(epochs):
        for data in feedback_loader:
            prompt, response = data["prompt"], data["response"]
            loss = model.compute_loss(prompt, response)  # directly use this as loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

# Here you'd provide the correct feedback
correct_response = "Product Development Team"
if prediction != correct_response:
    fine_tune_on_feedback(sentence, correct_response)

In [143]:
sentence = "There is some issue in the Home app. It stops as soon as it opens." # Product Development Team
prediction = model.predict(sentence)
print("Predicted Response:", prediction)

tensor([5.6736e-07, 9.1751e-09, 7.9027e-09,  ..., 9.5870e-09, 7.6453e-09,
        9.9951e-01])
50257
Predicted Response: Support Team


### Neural Network

In [129]:
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Recall

In [100]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_encoded = to_categorical(y_encoded) # One-hot encode the integer labels
X_encoded = list(use_model(X))

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.1, random_state=42)

In [153]:
model = Sequential()
model.add(Dense(512, input_dim=512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

In [154]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(np.array(X_train), np.array(y_train), epochs=15, batch_size=16, verbose=1)
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)

print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

Epoch 1/15
76/76 [==============================] - 0s 1ms/step - loss: 0.8903 - accuracy: 0.6650
Epoch 2/15
76/76 [==============================] - 0s 1ms/step - loss: 0.5747 - accuracy: 0.7980
Epoch 3/15
76/76 [==============================] - 0s 1ms/step - loss: 0.4459 - accuracy: 0.8362
Epoch 4/15
76/76 [==============================] - 0s 1ms/step - loss: 0.4051 - accuracy: 0.8454
Epoch 5/15
76/76 [==============================] - 0s 1ms/step - loss: 0.3626 - accuracy: 0.8570
Epoch 6/15
76/76 [==============================] - 0s 1ms/step - loss: 0.3534 - accuracy: 0.8728
Epoch 7/15
76/76 [==============================] - 0s 2ms/step - loss: 0.3477 - accuracy: 0.8803
Epoch 8/15
76/76 [==============================] - 0s 1ms/step - loss: 0.3019 - accuracy: 0.8820
Epoch 9/15
76/76 [==============================] - 0s 1ms/step - loss: 0.2867 - accuracy: 0.8919
Epoch 10/15
76/76 [==============================] - 0s 1ms/step - loss: 0.2963 - accuracy: 0.8828
Epoch 11/15
76/76 [

In [155]:
# Make predictions
y_pred = model.predict(np.array(X_test))
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Generate classification report
report = classification_report(y_test_labels, y_pred_labels, target_names=encoder.classes_)
print("Classification Report:")
print(report)

5/5 [==============================] - 0s 761us/step
Classification Report:
                          precision    recall  f1-score   support

            Finance Team       0.95      0.93      0.94        44
Product Development Team       0.90      0.85      0.88        55
            Support Team       0.74      0.83      0.78        35

                accuracy                           0.87       134
               macro avg       0.87      0.87      0.87       134
            weighted avg       0.88      0.87      0.87       134



## use the Roberta Method

In [161]:
from transformers import RobertaTokenizer, RobertaModel
import torch

In [162]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [164]:
X = df['Request']
y = df['Team']

In [165]:
# Encode sentences using RoBERTa
X_encoded = []
for sentence in X:
    inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    sentence_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    X_encoded.append(sentence_embedding)

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.1, random_state=42)

In [167]:
param_grid = {
    'C': [5, 6, 7, 8, 9],
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

In [168]:
classifier = SVC()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(classifier, param_grid, cv=cv, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [5, 6, 7, 8, 9], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'sigmoid']},
             scoring='accuracy')

In [169]:
best_params = grid_search.best_params_
print(best_params)
best_classifier = grid_search.best_estimator_

{'C': 7, 'gamma': 'scale', 'kernel': 'linear'}


In [170]:
y_pred = best_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Best Parameters: {best_params}")
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Best Parameters: {'C': 7, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 0.8656716417910447
Classification Report:
                          precision    recall  f1-score   support

            Finance Team       0.98      0.93      0.95        44
Product Development Team       0.88      0.84      0.86        55
            Support Team       0.72      0.83      0.77        35

                accuracy                           0.87       134
               macro avg       0.86      0.87      0.86       134
            weighted avg       0.87      0.87      0.87       134

